In [6]:
## import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc4
import tarfile
import json
import uuid as uuid
import os
from datetime import datetime, timedelta
from pathlib import Path
import s3fs
import argparse
from pprint import pprint
import time 
import gzip
import traceback
from contextlib import contextmanager

## import ECCO utils
import sys
sys.path.append('/home/jpluser/git_repos/ECCOv4-py')
import ecco_v4_py as ecco

In [14]:
def push_nc_dir_to_ec2(nc_root_dir_ec2, root_dest_s3_name):
    """
    Pushes the netcdf files from a directory to an S3 bucket.

    Args:
        nc_root_dir_ec2 (str): The root directory containing the netcdf files on the EC2 instance.
        root_dest_s3_name (str): The root name of the S3 bucket where the files will be pushed.

    Returns:
        None
    """
    ## upload each subdirectory in the nc root directory
    
    # Get a list of all items (files and folders) in the directory
    all_subdirs = os.listdir(nc_root_dir_ec2)

    for subdir_name in all_subdirs:
        if subdir_name != '.ipynb_checkpoints':
            print(subdir_name)
            
            ## push file to s3 bucket
            mybucket = root_dest_s3_name + subdir_name
            subdir_fullpath = Path(nc_root_dir_ec2 + "/" + subdir_name)
            nc_files = list(subdir_fullpath.glob('*.nc'))
    
            print(f'\n>pushing netcdf files in {subdir_name} to s3 bucket : {mybucket}')
            print(f'... looking for *.nc files in {subdir_name}')
            print(f'... found {len(nc_files)} nc files to upload')
    
            if len(nc_files)>0:
                cmd=f"aws s3 cp {subdir_fullpath} {mybucket}/ --recursive --include '*.nc' --no-progress > /dev/null 2>&1"
                print(f'... aws command: {cmd}')
                with suppress_stdout():
                   os.system(cmd)
            else:
                print("... nothing to upload!") 
            
        else:
            print(f"No subdirectories found in {nc_root_dir_ec2}")

In [15]:
nc_root_dir_ec2 = '/home/jpluser/git_repos/SASSIE_downscale_ecco_v5/data/tmp_nc/tr_diff_x_day_mean_0005790000'
root_dest_s3_name = 's3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/'

In [16]:
push_nc_dir_to_ec2(nc_root_dir_ec2, root_dest_s3_name)

DFxE_SLT_AVG_DAILY

>pushing netcdf files in DFxE_SLT_AVG_DAILY to s3 bucket : s3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/DFxE_SLT_AVG_DAILY
... looking for *.nc files in DFxE_SLT_AVG_DAILY
... found 5 nc files to upload
... aws command: aws s3 cp /home/jpluser/git_repos/SASSIE_downscale_ecco_v5/data/tmp_nc/tr_diff_x_day_mean_0005790000/DFxE_SLT_AVG_DAILY s3://podaac-dev-sassie/ECCO_model/N1/V1/HH/NETCDF/DFxE_SLT_AVG_DAILY/ --recursive --include '*.nc' --no-progress > /dev/null 2>&1


NameError: name 'suppress_stdout' is not defined